In [ ]:
#Imports
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
import os
sns.set(style = "darkgrid")

In [ ]:
from functions import con_data
from functions import lable_loading #load labels
from functions import frame_gen #data frame generator
from functions import index_gen #index generator
from functions import tr_idx_updater #training index updater

In [ ]:
#parameters

SEED = 12
image_height = 128
image_width = 128
dimention = (image_height, image_width)
Training_set= 10000
# Training_set= 10000
Validation_set= 3000
# Validation_set= 3000
EPOCHS = 2
BS = 32
n = 5000
# n = 50000
Learning_rate = 0.0001
label_file_train = "/home/pramod/COVID-19_Detection/image/archive/train_COVIDx_CT-2A.txt"
# print("label_file_train",label_file_train[: 40])
label_file_valid = "/home/pramod/COVID-19_Detection/image/archive/val_COVIDx_CT-2A.txt"

## LOADING DATA

In [ ]:
#%%%%%%%%%%%%%%%%%%%%%%%%
# loading the data
#%%%%%%%%%%%%%%%%%%%%%%%%
file_names_train , classes_train ,bboxes_train = lable_loading(label_file_train)
# print("file_names_train",file_names_train[: 20])
file_names_valid , classes_valid ,bboxes_valid = lable_loading(label_file_valid)
train_index = index_gen(file_names_train , Training_set)
valid_index = index_gen(file_names_valid , Validation_set)
train_index_updated = tr_idx_updater(classes_train,train_index,n )
df = frame_gen(train_index_updated , valid_index , classes_train , classes_valid)
df.plot.bar( title = "Data Image Distribution",color=['green', 'blue'] );
plt.savefig('data.png', bbox_inches='tight')

## Loading Training and Validation Images

In [ ]:

x_train, y_train = con_data(file_names_train,classes_train, dimention , index  =train_index_updated ,bboxes = bboxes_train )
x_valid , y_valid= con_data(file_names_valid,classes_valid, dimention , index  =valid_index , bboxes = bboxes_valid)

## Pre-Processing

In [ ]:
x_train = tf.keras.applications.densenet.preprocess_input(x_train)
x_valid = tf.keras.applications.densenet.preprocess_input(x_valid)

In [ ]:
for i in range(len(y_train)):
    if y_train[i] ==1:
        y_train[i]=0
    if y_train[i] ==2:
        y_train[i]=1
for i in range(len(y_valid)):
    if y_valid[i] ==1:
        y_valid[i]=0
    if y_valid[i]==2:
        y_valid[i]=1

In [ ]:
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,zoom_range=0.05,width_shift_range=0.2 ,height_shift_range=0.1,shear_range=0.05)## ENTER YOUR AUG HERE
train_generator = data_generator.flow(x_train, y_train, BS)
spe = x_train.shape[0] // BS

## MODEL DESIGN

In [ ]:
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input



In [ ]:
base_model = DenseNet201(include_top=False,weights="imagenet",input_shape=(image_height,image_height,3))
model= tf.keras.Sequential()
model.add(base_model) 
model.add(Dense(64,activation=('relu'))) 
model.add(GlobalAveragePooling2D())
model.add(Dense(1,activation=('sigmoid')))
model.add(Dropout(.2))


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(Learning_rate),
              loss = "binary_crossentropy",
              metrics = ["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
modelsdir = "models"
checkpoint = ModelCheckpoint(filepath = modelsdir + "/SGD.{epoch:02d}-{val_accuracy:.4f}.h5" ,
                             monitor= "val_accuracy" , 
                             mode = "max",
                            verbose = 0,
                            save_best_only=True)

In [ ]:
r = model.fit(train_generator , validation_data=(x_valid, y_valid), steps_per_epoch = spe ,epochs=EPOCHS, callbacks = [checkpoint] ,verbose = 2 )

In [ ]:
plt.plot(r.history["loss"], label= "Loss")
plt.plot(r.history["val_loss"], label = "val_Loss")
plt.legend();
plt.savefig('loss.png', bbox_inches='tight')

In [ ]:
plt.plot(r.history["accuracy"] ,label = "Accuracy")
plt.plot(r.history["val_accuracy"] ,label = "Val_Accuracy")
plt.legend();
plt.savefig('accuracy.png', bbox_inches='tight')

In [ ]:
pd.DataFrame(r.history).to_csv("densenet.csv",index=False)

In [ ]:
model.save("Densenet.h5")

## MORE TRAINING

In [ ]:
Learning_rate = 0.001
EPOCHS = 25

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(Learning_rate),
              loss = "binary_crossentropy",
              metrics = ["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
modelsdir = "models"
checkpoint = ModelCheckpoint(filepath = modelsdir + "/(mt)SGD.{epoch:02d}-{val_accuracy:.4f}.h5" ,
                             monitor= "val_accuracy" , 
                             mode = "max",
                            verbose = 0,
                            save_best_only=True)

In [ ]:
s = model.fit(train_generator , validation_data=(x_valid, y_valid), steps_per_epoch = spe ,epochs=EPOCHS, callbacks = [checkpoint] ,verbose = 2 )

In [ ]:
plt.plot(s.history["loss"], label= "Loss")
plt.plot(s.history["val_loss"], label = "val_Loss")
plt.legend();
plt.savefig('e_loss.png', bbox_inches='tight')

In [ ]:
plt.plot(s.history["accuracy"] ,label = "Accuracy")
plt.plot(s.history["val_accuracy"] ,label = "Val_Accuracy")
plt.legend();
plt.savefig('e_accuracy.png', bbox_inches='tight')

In [ ]:
pd.DataFrame(s.history).to_csv("densenet-2.csv",index=False)

In [ ]:
model.save("Densenet-2.h5")